In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))


In [4]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [5]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [6]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [8]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [9]:
# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(50, name='lstm_layer'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax', name='output_layer'))

In [10]:
# Compile the model initially
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Iterative layer freezing
for i in range(len(lstm_model.layers)):
    # Freeze all layers up to the current layer
    for j in range(i + 1):
        lstm_model.layers[j].trainable = False

    # Re-compile the model to apply changes
    lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print(f"Training with layers 0 to {i} frozen...")

    # Train model for a few epochs
    lstm_model.fit(X_train_lstm, y_train, epochs=5, batch_size=32, validation_data=(X_test_lstm, y_test))

    # Evaluate the model on the validation set
    val_loss, val_accuracy = lstm_model.evaluate(X_test_lstm, y_test)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

Training with layers 0 to 0 frozen...
Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2863 - loss: 1.6630 - val_accuracy: 0.2857 - val_loss: 1.6222
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3578 - loss: 1.5486 - val_accuracy: 0.3333 - val_loss: 1.5591
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3694 - loss: 1.5031 - val_accuracy: 0.3757 - val_loss: 1.5049
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4115 - loss: 1.4631 - val_accuracy: 0.4233 - val_loss: 1.4561
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4454 - loss: 1.4032 - val_accuracy: 0.4868 - val_loss: 1.4118
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5066 - loss: 1.3832 
Validation Loss: 1.4117778539657593, Validation Accuracy: 0.48677247762680054
Training with layers 0 to 1 frozen...
Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4893 - loss: 1.3567 - val_accuracy: 0.5079 - val_loss: 1.3669
Epoch 2/5
24/24 ━━━━━━

C:\Users\HP\anaconda3\envs\mediapipe\lib\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5573 - loss: 1.1899 - val_accuracy: 0.5503 - val_loss: 1.2061
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6219 - loss: 1.1654 - val_accuracy: 0.5503 - val_loss: 1.2061
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6036 - loss: 1.1621 - val_accuracy: 0.5503 - val_loss: 1.2061
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6392 - loss: 1.1569 - val_accuracy: 0.5503 - val_loss: 1.2061
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5993 - loss: 1.1890 - val_accuracy: 0.5503 - val_loss: 1.2061
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5903 - loss: 1.1780 
Validation Loss: 1.2061306238174438, Validation Accuracy: 0.5502645373344421


In [12]:
# After loop, unfreeze all layers for final fine-tuning
for layer in lstm_model.layers:
    layer.trainable = True

In [13]:
# Compile the model again for fine-tuning
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Perform fine-tuning
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6379 - loss: 1.0891 - val_accuracy: 0.7037 - val_loss: 0.9941
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7490 - loss: 0.9410 - val_accuracy: 0.7143 - val_loss: 0.8472
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7728 - loss: 0.8309 - val_accuracy: 0.8360 - val_loss: 0.7539
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8179 - loss: 0.7627 - val_accuracy: 0.8730 - val_loss: 0.6935
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8530 - loss: 0.6749 - val_accuracy: 0.9048 - val_loss: 0.6195
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8923 - loss: 0.6084 - val_accuracy: 0.8942 - val_loss: 0.5857
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9037 - loss: 0.5727 - val_accuracy: 0.9101 - val_loss: 0.5297
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9110 - loss: 0.5235 - val_accuracy: 0.8942 - val_loss

In [15]:
# Save the training history
with open('training_history_loop.json', 'w') as f:
    json.dump(lstm_history.history, f)

In [16]:
# Save the final LSTM model
lstm_model.save('verification_model_loop.h5')